In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import google.colab

google.colab.drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!pip install pyarrow
!pip install snappy
!pip install fastparquet
!pip install python-snappy
!sudo apt-get install libsnappy-dev

  Using cached https://files.pythonhosted.org/packages/45/35/65d9f8cc537129894b4b32647d80212d1fa342877581c5b8a69872cea8be/python-snappy-0.5.4.tar.gz
  Created wheel for python-snappy: filename=python_snappy-0.5.4-cp36-cp36m-linux_x86_64.whl size=40382 sha256=a0d74b0ab91331b3dea59873ac40fdbadd0b5fe8f0f20c865f72e0c9835f80c3
  Stored in directory: /root/.cache/pip/wheels/df/aa/d5/459b834baa4c9f0ea918a38750fb29981e4a01ef523a936c06
Successfully built python-snappy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsnappy-dev is already the newest version (1.1.7-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [3]:
DATADIR = "/content/drive/My Drive/lenta_data"

In [ ]:
df = pd.read_parquet(f"{DATADIR}/transactions.parquet", engine='fastparquet', columns=["client_id", "material"])
df.sample(5)

,client_id,material
16892688,2965cf80d9bbb20a17f18e225cc936f3,ef60aa64492347ae69e7c9309db3aea0
7226090,a62e2abece8fa4aa04220aa698de7da6,e7d09ab7a6ee5037832158dc0b9a4454
11599282,7f3c363f1cc59cc29d8eda446175ea69,372754b057c09ed0cb0eebdfa8d5ee9d
26263794,e6fc1e0050ca6b449d328c8fda6852c1,6278f48a43fb0a23036b3a11c92e726c
9230135,3c64a7e2a05f74ca7212080191a32ba8,b75147feae4cf8ec07fb5f2cef855def


In [ ]:
# remove returns

df = df[df['sales_count'] > 0]
df.sample(5)

KeyError: ignored

In [ ]:
materials_df = pd.read_csv(f"{DATADIR}/materials.csv", usecols=['material', 'hier_level_4'])
materials_df.sample(5)

In [ ]:
df = df.merge(materials_df, on=['material'], how='left')
df.sample(5)

### Material sales per customer

In [ ]:
mat_sales_df = df.groupby(['client_id', 'material'])['sales_count'].sum().to_frame()
mat_sales_df.sample(5)

,,sales_count
client_id,material,
a42af8fed56ec357e977cd5177cbda28,696fbc071fe38ad312de354ada7db110,1.0
cba3e840d3a7a44137cc670c82afb206,fde94cc3d8cf54eac865417c18964886,1.0
eb64af74c2384f14c1ce7ed710120c6d,4e0bfc102b2afe74eb8eec334bfd5672,1.0
ae378f5233a03c980dfae444a5d3a018,c2fcf6111267657bda916456b1492244,1.0
f24929349161201d3ca6947c2da0f0d0,3c9ad1d7af2b544ed46c98b8710292d2,6.0


In [ ]:
mat_sales_df.reset_index(level=[0,1], inplace=True)
mat_sales_df.rename(columns={'sales_count' : 'mat_sales'}, inplace=True)
mat_sales_df.sample(5)

,client_id,material,mat_sales
1843223,17c4ad7aa626f5051a9a93f9615f3647,7e69100f4f3a58f1b9b37bafca74bce1,0.142
19323489,f5ebe737c958a2d1775aa226fd0d085c,ca89bab55eddbf6c3b9c833f9290ca7d,1.000
8936620,72eceee7559e43baeca91548c48a5884,3011e8bf66f06db80b1ad407c631a4e9,1.000
994718,0ce99d6bad1c3b3ed3e3e994ba40ea54,e5ac6002d3cf6f84095fb58fcfa5136c,1.000
15299787,c38972aacdae98fc2944383704e03fbb,f8c8aa3854a16222fc785c53de60d31d,1.000


In [ ]:
mat_sales_df.to_csv(f"{DATADIR}/mat_sales.csv", index=False)

### Category popularity per customer

In [ ]:
client_category_df = df.groupby(['client_id', 'hier_level_4']).count().to_frame()
client_category_df.sample(5)

,,material
client_id,hier_level_4,
8422d1286e7450e049153a8a813f8ecb,86f69250cc4aceacb5e7a6e65db3768a,1
98c1d9211a6c1ffc15126f43adb3d018,ffa5a75bfd08cbe4a6a3b3328d5876ac,1
68cee1c06fe8657eb8fe94a2be64c7e2,b7fa186535a75c869bfaece622c72689,1
b34c42102e05d659b62fd3728fbd2ab8,9398d4c1ebb5d3e4f09550507c17f4de,6
2c924601c8ca3ea5f1bd74de5160cda3,2e4e106f70093bd3ef9b9d4c62acc14f,1


In [ ]:
client_material_df.reset_index(level=1, inplace=True)
client_material_df['cat_count'] = client_material_df['chq_date']
client_material_df.drop(columns=['chq_date', 'sales_count'], inplace=True)

In [ ]:
client_category_df = client_category_df.reset_index(level=1)
client_category_df.sample(5)

,hier_level_4,material
client_id,,
0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,2
0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2
0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,2
0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,1
0000d48a8891cd462fdf01ea861ec53e,0cf00eeda915a979c5b07f6904942c2a,2
...,...,...
fffd3e78e3ea6bf00aee1372efc09dad,d66a5517fea10ab5d81ee32912ef07e6,1
fffd3e78e3ea6bf00aee1372efc09dad,dd1543656cc988aac30ead1562ec6e4d,1
fffd3e78e3ea6bf00aee1372efc09dad,e817ace3fff08334e8a7a9a582078ce4,1


In [ ]:
client_sums = client_category_df.groupby('client_id')['material'].sum().to_frame()
client_sums.sample(5)

,material
client_id,
4bf1b6b0a43c2214def7e53389a8feca,68
822fdb19132d3b5e4d72c8c2d6af82ec,59
1c2e6f96592bd585e10aac27928b3617,204
0979d5d06a4a394c1d5281e88a24c913,434
65e02eee1cce93daf9f70ba412082977,123


In [ ]:
client_category_df = client_category_df.merge(client_sums, on=['client_id'], how='left')
client_category_df['material_x'] = client_category_df['material_x'] / client_category_df['material_y']
client_category_df.rename(columns={'material_x' : 'cat_pop'}, inplace=True)
client_category_df.drop(columns=['material_y'], inplace=True)
client_category_df.sample(5)

,hier_level_4,cat_pop
client_id,,
8436435695240955d590b7bb699119f6,b0327e81a6d925aaefbac4b5e46a2313,0.005291
7df2ef97e26d252800b1f622b27faac0,8ce911d01167e72108017ec043de6352,0.007937
8953013058a0c796903f208ca1605ac6,d88670481737b727c6dedc2cd9f8eafd,0.002625
aa5f921ac6b764c41c8eabc105377d23,e4215837d5df119daffad3b7e5bf5ad9,0.007299
f0065184b3d8161f7450d16b39b619c7,3a3943f6ddf828e4a605c3a1ddbd7859,0.001718


In [ ]:
client_category_df.reset_index(level=0, inplace=True)
client_category_df.to_csv(f"{DATADIR}/client_cat_pop.csv", index=False)
client_category_df.sample(5)

,client_id,hier_level_4,cat_pop
8957680,ca88ff0b6963966b4994a8846da93d10,01600503c4ba899b52ed923c5ab88e20,0.009901
7071038,a040a5a63e3d5f09457d5106faad698d,34e3d6f45aa01f0c59531c5568bf8ec9,0.001883
8189558,b966c7ba2ed53aef6bdb2f03dceca7f1,113d428f6e00d3c6420220df16c473c9,0.003937
6989059,9e5719421db12c3e71a553533ac91718,469b0c89135a359fc5729312490bfa08,0.017647
2117869,2fe32579b99e2c879352fcc4b3eb2955,8d64a17e811aeae89de1a031b1d2d7c4,0.001647


### In-category popularity of materials per customer

In [ ]:
client_material_df = df.groupby(['client_id', 'hier_level_4', 'material']).count()
client_material_df.sample(5)

,,,chq_date,sales_count
client_id,hier_level_4,material,,
34502efc7781b37f9145cefbd16b10af,3f6ab5dd77f73b11f122ac7c0fe2de76,723dcffa19fd20048bfcb34a595592da,7,7
b074b4460d2f784076a230fd30e93341,be7712274ca785f4c41aa0eab4ffe09a,f8d64eea9f558a3d41b21d9562d7358c,1,1
c4f5db5191bfb3cad555bbcb9b5c42b9,17bdeb073cbf020103735133de8927f8,ca46a04b54077c9d321bc33eb4f4adc1,2,2
25828dd63e5703156a0be779bd1cc57d,7b37ae98d07f7b0fa5c8a6bd60694747,dc5ba1afc34d39cf5e131c122cc58d5d,1,1
a5da38d130e8dabc37d9c595f0f9e76a,fb4f098d94dd7fd40311020ee5b1bff4,182999808e8e8ce0b35df860c4dce872,4,4


In [ ]:
client_material_df.reset_index(level=[1,2], inplace=True)
client_material_df['material_count'] = client_material_df['chq_date']
client_material_df.drop(columns=['chq_date', 'sales_count'], inplace=True)
client_material_df.sample(5)

,hier_level_4,material,material_count
client_id,,,
5f515b67072402af7c24ab3be5ccae55,3f6ab5dd77f73b11f122ac7c0fe2de76,ebae2dec1bcb8b456f983d0747b8db2d,2
51804ff9a86d8609b9b39ba3f9b582f1,3809a8eb9a716dc7a67dee931264e096,82794b04013dcd0fc7fafbdc2f98f69b,1
d80a1b9c924bc42554c68a17e9440979,e49bcfc60cfc2c4f3ef522bd4761bce5,71b11eaf018606a4d5157f3c7d2d4b25,1
6ff3a15579422854317d93c3f014eed6,6be199d6eb8d811805f8df9811842560,23f81ecaf61cde66f2bf2f8e308917ef,1
27561ac9413263c21e693f32fbb5787d,3f6ab5dd77f73b11f122ac7c0fe2de76,cdab3403355b7fda81a1329b2f05cdec,1


In [ ]:
client_hier_cap = df.groupby(['client_id', 'hier_level_4']).count()
client_hier_cap.reset_index(level=1, inplace=True)
client_hier_cap['cat_count'] = client_hier_cap['chq_date']
client_hier_cap.drop(columns=['chq_date', 'sales_count', 'material'], inplace=True)
client_hier_cap.sample(5)

,hier_level_4,cat_count
client_id,,
ff81f196c7a46f50715a0492b2ddd4c8,6dc943f2c2dcfd0644a1a28562f2befc,2
4fd4b372c45d2752c9dc095618d6427a,f8785dbb7fd142e7a56b41f87a81535d,1
969b1d22f171ed63c75f24eb7e221bea,fb8c9cd692aba925faf07d40c37f1f3b,4
a29daf88b13fb7f986b527a4a7bacd7f,17bdeb073cbf020103735133de8927f8,5
21c83d4db0e4722624216fe922ab9539,38649fe46f1c4db7c7a53c780b363a77,2


In [ ]:
client_material_df = client_material_df.merge(client_hier_cap, on=['client_id', 'hier_level_4'], how='left')
client_material_df.sample(5)

,hier_level_4,material,material_count,cat_count
client_id,,,,
7f0c3be5e84b9ba649937d7d69842c3a,65ce9a05680f7ff3339a68cc1eac2330,3b1e55c2c0adf65ae25d1e7072f82d58,1,4
7899016e3fafdc0a775d23431abd6aab,88bc8e4c598d233294385b72836ab725,eed52379ce1812c0aee6332a87a68d6b,1,4
483754ceb47125304911fdf7d67e70e0,b0224487031005a7276adfc250b0a2a0,dff8245269fc07958946b34117805cba,1,1
92119d3cab6b9432e6bca71624ae0e16,cac4a6f04c1e60ff2bc0fd4b1aa62bfa,ed949db7b783c41711f47f104ea495f2,1,1
18651e232b4a7ae6ddff47b451c3ea0b,e36062571db5523ef17e546b7dd14e9d,fc3903da66ffc4070fbcda0ed83459c9,1,3


In [ ]:
client_material_df['mat_loyalty'] = client_material_df['material_count'] / client_material_df['cat_count']
client_material_df.sample(5)

,hier_level_4,material,material_count,cat_count,mat_loyalty
client_id,,,,,
14c4564c48cd83957b0347752c89bae6,6aee43d34c0c1d275ea483d0a5a0f069,7ff4d25fcbdba033c06e2ba25ae6fc77,1,1,1.000000
bfbb37157fcd77431cd2cdc6adacbbd3,879848e74e5c4ad8d310bd70bbdce0a5,fb54b7192cbbed520103ba25501cb804,1,2,0.500000
045f616846d1db95bf9f2d4c2d529239,20ef90b7cafa2df5c24823b7c1d5ca45,995f7fafcf86076481519094a195c852,1,2,0.500000
a05a27a48856a7be6257446886051c08,32d4d8df25a4d20cc801336137732213,17e955baa18e441dbf74a009516c95e2,2,61,0.032787
34b40e8f8530836ab0ca5dfe2735be1f,a51bf61acca38b2a1c36aeef35a43cf5,9db7f0e4cda2082db9cdf462b7efff00,1,4,0.250000


In [ ]:
client_material_df.reset_index(level=0)

,client_id,hier_level_4,material,material_count,cat_count,mat_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,33306ccd877e73c388652739869d3dc7,2,3,0.666667
1,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,a5f9a709dcfdbfa96bd9c2ed60aa194f,1,3,0.333333
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2f84dcde18dbe95facff8ec9bcf91e0d,2,3,0.666667
3,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,590b04ede14c4b6955326709d8852853,1,3,0.333333
4,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,4,5,0.800000
...,...,...,...,...,...,...
20107271,fffd3e78e3ea6bf00aee1372efc09dad,d66a5517fea10ab5d81ee32912ef07e6,f27d1e86a3fdddc3efea8d9b460376aa,1,1,1.000000
20107272,fffd3e78e3ea6bf00aee1372efc09dad,dd1543656cc988aac30ead1562ec6e4d,ccb92be59ce7e5f3371b0dea5e871a9b,1,1,1.000000
20107273,fffd3e78e3ea6bf00aee1372efc09dad,e817ace3fff08334e8a7a9a582078ce4,34f2c15a8df21b347d939ac0d762798d,1,1,1.000000
20107274,fffd3e78e3ea6bf00aee1372efc09dad,f1471bc23e88cb9d971953f99e020188,be497986bd5afd60acc5b5b0e7bdfcee,1,1,1.000000


In [ ]:
client_material_df.reset_index(level=0, inplace=True)
client_material_df.to_csv(f"{DATADIR}/client_mat.csv", index=False)

### Combine two

In [ ]:
client_cat_df = pd.read_csv(f"{DATADIR}/client_cat_pop.csv")
client_cat_df.head(5)

,client_id,hier_level_4,cat_pop
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,0.005405
1,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.005405
2,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,0.005405
3,0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,0.002703
4,0000d48a8891cd462fdf01ea861ec53e,0cf00eeda915a979c5b07f6904942c2a,0.005405


In [ ]:
client_mat_df = pd.read_csv(f"{DATADIR}/client_mat.csv")
client_mat_df.head(5)

,client_id,hier_level_4,material,material_count,cat_count,mat_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,33306ccd877e73c388652739869d3dc7,2,3,0.666667
1,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,a5f9a709dcfdbfa96bd9c2ed60aa194f,1,3,0.333333
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2f84dcde18dbe95facff8ec9bcf91e0d,2,3,0.666667
3,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,590b04ede14c4b6955326709d8852853,1,3,0.333333
4,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,4,5,0.800000


In [ ]:
client_mat_df.drop(columns=['material_count', 'cat_count'], inplace=True)
client_mat_df.head(5)

,client_id,hier_level_4,material,mat_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,33306ccd877e73c388652739869d3dc7,0.666667
1,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,a5f9a709dcfdbfa96bd9c2ed60aa194f,0.333333
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2f84dcde18dbe95facff8ec9bcf91e0d,0.666667
3,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,590b04ede14c4b6955326709d8852853,0.333333
4,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,0.800000


In [ ]:
client_mat_df = client_mat_df.merge(client_cat_df, on=['client_id', 'hier_level_4'], how='left')
client_mat_df.head(5)

,client_id,hier_level_4,material,mat_loyalty,cat_pop
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,33306ccd877e73c388652739869d3dc7,0.666667,0.005405
1,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,a5f9a709dcfdbfa96bd9c2ed60aa194f,0.333333,0.005405
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2f84dcde18dbe95facff8ec9bcf91e0d,0.666667,0.005405
3,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,590b04ede14c4b6955326709d8852853,0.333333,0.005405
4,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,0.800000,0.005405


In [ ]:
client_mat_df['cum_loyalty'] = 2 * client_mat_df['mat_loyalty'] * client_mat_df['cat_pop'] / (client_mat_df['mat_loyalty'] + client_mat_df['cat_pop'])
client_mat_df.head(5)

,client_id,hier_level_4,material,mat_loyalty,cat_pop,cum_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,33306ccd877e73c388652739869d3dc7,0.666667,0.005405,0.010724
1,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,a5f9a709dcfdbfa96bd9c2ed60aa194f,0.333333,0.005405,0.010638
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2f84dcde18dbe95facff8ec9bcf91e0d,0.666667,0.005405,0.010724
3,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,590b04ede14c4b6955326709d8852853,0.333333,0.005405,0.010638
4,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,0.800000,0.005405,0.010738


In [ ]:
client_weights_df = pd.read_csv(f"{DATADIR}/client_weights.csv", usecols=['client_id', 'weight'])
client_weights_df.head(5)

,client_id,weight
0,0000d48a8891cd462fdf01ea861ec53e,0.80
1,00028a1dcb30863404a386961a69300e,0.65
2,0004274683494a9eca5e70ef97299d55,0.70
3,00072f81efcb3481861f7300c5e2a88a,0.20
4,00076596eb749cf10c2e984621dffe95,0.85


In [ ]:
client_mat_df = client_mat_df.merge(client_weights_df, on=['client_id'], how='left')
client_mat_df.head(5)

,client_id,hier_level_4,material,mat_loyalty,cat_pop,cum_loyalty,weight
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,33306ccd877e73c388652739869d3dc7,0.666667,0.005405,0.010724,0.8
1,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,a5f9a709dcfdbfa96bd9c2ed60aa194f,0.333333,0.005405,0.010638,0.8
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2f84dcde18dbe95facff8ec9bcf91e0d,0.666667,0.005405,0.010724,0.8
3,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,590b04ede14c4b6955326709d8852853,0.333333,0.005405,0.010638,0.8
4,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,0.800000,0.005405,0.010738,0.8


In [ ]:
client_mat_df['tot_loyalty'] = client_mat_df['cum_loyalty'] * client_mat_df['weight']
client_mat_df.head(5)

,client_id,hier_level_4,material,mat_loyalty,cat_pop,cum_loyalty,weight,tot_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,33306ccd877e73c388652739869d3dc7,0.666667,0.005405,0.010724,0.8,0.008579
1,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,a5f9a709dcfdbfa96bd9c2ed60aa194f,0.333333,0.005405,0.010638,0.8,0.008511
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,2f84dcde18dbe95facff8ec9bcf91e0d,0.666667,0.005405,0.010724,0.8,0.008579
3,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,590b04ede14c4b6955326709d8852853,0.333333,0.005405,0.010638,0.8,0.008511
4,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,692ac9d604f2d7c4142c6e922e4def19,0.800000,0.005405,0.010738,0.8,0.008591


In [ ]:
client_mat_df.to_csv(f"{DATADIR}/client_loyalty_fin.csv", index=False)

### Material loyalty ranking

In [4]:
mat_loyal_df = pd.read_csv(f"{DATADIR}/loyalty_totals.csv")
mat_loyal_df.head(5)

,Unnamed: 0,client_id,hier_level_4,loyalty,material,sales_count,cat_loyalty,harmonic_mean_loyalty
0,0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,0.681679,33306ccd877e73c388652739869d3dc7,1.786,0.002713,0.005404
1,1,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,2f84dcde18dbe95facff8ec9bcf91e0d,2.000,0.004142,0.008216
2,2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,590b04ede14c4b6955326709d8852853,2.000,0.004142,0.008216
3,3,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,0.761905,692ac9d604f2d7c4142c6e922e4def19,16.000,0.021745,0.042284
4,4,0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,1.000000,b619edd00691c83bb45d441cacbb7a99,0.866,0.000897,0.001792


In [5]:
mat_loyal_df.drop(columns=['Unnamed: 0'], inplace=True)
mat_loyal_df.head(5)

,client_id,hier_level_4,loyalty,material,sales_count,cat_loyalty,harmonic_mean_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,0.681679,33306ccd877e73c388652739869d3dc7,1.786,0.002713,0.005404
1,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,2f84dcde18dbe95facff8ec9bcf91e0d,2.000,0.004142,0.008216
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,590b04ede14c4b6955326709d8852853,2.000,0.004142,0.008216
3,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,0.761905,692ac9d604f2d7c4142c6e922e4def19,16.000,0.021745,0.042284
4,0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,1.000000,b619edd00691c83bb45d441cacbb7a99,0.866,0.000897,0.001792


In [6]:
client_weights_df = pd.read_csv(f"{DATADIR}/client_weights.csv", usecols=['client_id', 'weight'])
client_weights_df.head(5)

,client_id,weight
0,0000d48a8891cd462fdf01ea861ec53e,0.80
1,00028a1dcb30863404a386961a69300e,0.65
2,0004274683494a9eca5e70ef97299d55,0.70
3,00072f81efcb3481861f7300c5e2a88a,0.20
4,00076596eb749cf10c2e984621dffe95,0.85


In [7]:
mat_loyal_df = mat_loyal_df.merge(client_weights_df, on=['client_id'], how='left')
mat_loyal_df.head(5)

,client_id,hier_level_4,loyalty,material,sales_count,cat_loyalty,harmonic_mean_loyalty,weight
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,0.681679,33306ccd877e73c388652739869d3dc7,1.786,0.002713,0.005404,0.8
1,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,2f84dcde18dbe95facff8ec9bcf91e0d,2.000,0.004142,0.008216,0.8
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,590b04ede14c4b6955326709d8852853,2.000,0.004142,0.008216,0.8
3,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,0.761905,692ac9d604f2d7c4142c6e922e4def19,16.000,0.021745,0.042284,0.8
4,0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,1.000000,b619edd00691c83bb45d441cacbb7a99,0.866,0.000897,0.001792,0.8


In [144]:
mat_loyal_df['cum_loyalty'] = (1 + 4**2) * mat_loyal_df['loyalty'] * mat_loyal_df['cat_loyalty'] / ( 4**2 * mat_loyal_df['cat_loyalty'] + mat_loyal_df['loyalty'] )
mat_loyal_df.head(5)

,client_id,hier_level_4,loyalty,material,sales_count,cat_loyalty,harmonic_mean_loyalty,weight,tot_loyalty,weighted_sales_count,cum_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,0.681679,33306ccd877e73c388652739869d3dc7,1.786,0.002713,0.005404,0.8,0.051323,0.8,0.043359
1,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,2f84dcde18dbe95facff8ec9bcf91e0d,2.000,0.004142,0.008216,0.8,0.071372,0.8,0.062173
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,590b04ede14c4b6955326709d8852853,2.000,0.004142,0.008216,0.8,0.071372,0.8,0.062173
3,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,0.761905,692ac9d604f2d7c4142c6e922e4def19,16.000,0.021745,0.042284,0.8,0.263960,0.8,0.253780
4,0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,1.000000,b619edd00691c83bb45d441cacbb7a99,0.866,0.000897,0.001792,0.8,0.018243,0.8,0.015029


In [145]:
mat_loyal_df['tot_loyalty'] = mat_loyal_df['cum_loyalty'] * mat_loyal_df['weight'] * mat_loyal_df['sales_count']
mat_loyal_df.head(5)

,client_id,hier_level_4,loyalty,material,sales_count,cat_loyalty,harmonic_mean_loyalty,weight,tot_loyalty,weighted_sales_count,cum_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,0.681679,33306ccd877e73c388652739869d3dc7,1.786,0.002713,0.005404,0.8,0.061952,0.8,0.043359
1,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,2f84dcde18dbe95facff8ec9bcf91e0d,2.000,0.004142,0.008216,0.8,0.099476,0.8,0.062173
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,590b04ede14c4b6955326709d8852853,2.000,0.004142,0.008216,0.8,0.099476,0.8,0.062173
3,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,0.761905,692ac9d604f2d7c4142c6e922e4def19,16.000,0.021745,0.042284,0.8,3.248382,0.8,0.253780
4,0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,1.000000,b619edd00691c83bb45d441cacbb7a99,0.866,0.000897,0.001792,0.8,0.010412,0.8,0.015029


In [146]:
mat_loyal_df['weighted_sales_count'] = mat_loyal_df['weight'] * mat_loyal_df['sales_count']
mat_loyal_df.head(5)

,client_id,hier_level_4,loyalty,material,sales_count,cat_loyalty,harmonic_mean_loyalty,weight,tot_loyalty,weighted_sales_count,cum_loyalty
0,0000d48a8891cd462fdf01ea861ec53e,001d2d283b7f3c7378e363313b483e51,0.681679,33306ccd877e73c388652739869d3dc7,1.786,0.002713,0.005404,0.8,0.061952,1.4288,0.043359
1,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,2f84dcde18dbe95facff8ec9bcf91e0d,2.000,0.004142,0.008216,0.8,0.099476,1.6000,0.062173
2,0000d48a8891cd462fdf01ea861ec53e,002fc5b5b4131c6ea6412ec21999589e,0.500000,590b04ede14c4b6955326709d8852853,2.000,0.004142,0.008216,0.8,0.099476,1.6000,0.062173
3,0000d48a8891cd462fdf01ea861ec53e,05d72f9cf2bd35d8e22a38d8045dcc7f,0.761905,692ac9d604f2d7c4142c6e922e4def19,16.000,0.021745,0.042284,0.8,3.248382,12.8000,0.253780
4,0000d48a8891cd462fdf01ea861ec53e,0c65d833d6d555e2702eb6da0b60f8c3,1.000000,b619edd00691c83bb45d441cacbb7a99,0.866,0.000897,0.001792,0.8,0.010412,0.6928,0.015029


In [147]:
mat_df = mat_loyal_df.groupby(['material'])['tot_loyalty'].sum().to_frame()
mat_df.head(5)

,tot_loyalty
material,
000062284ffb915ba6b88a36e3cb7ba8,1.424395
00011c1252f877221e0e0a3ddf21858e,9.962606
000189cbec3b82946747db131d67805a,12.108905
000273bf6eb4ba9a2ec52c0b5ee564ed,102.706557
00033dd12618c86f7451b1e18af8bb60,3.964742


In [148]:
mat_df['weights_sum'] = mat_loyal_df.groupby(['material'])['weighted_sales_count'].sum()
mat_df.rename(columns={'tot_loyalty' : 'tot_loyalty_sum'}, inplace=True)
mat_df.head(5)

,tot_loyalty_sum,weights_sum
material,,
000062284ffb915ba6b88a36e3cb7ba8,1.424395,21.25000
00011c1252f877221e0e0a3ddf21858e,9.962606,155.25000
000189cbec3b82946747db131d67805a,12.108905,137.80000
000273bf6eb4ba9a2ec52c0b5ee564ed,102.706557,1424.40415
00033dd12618c86f7451b1e18af8bb60,3.964742,87.75000


In [149]:
eps = 1e-8

In [150]:
mat_df['fin_loyalty'] = mat_df['tot_loyalty_sum'] / (mat_df['weights_sum'] + eps)
mat_df.head(5)

,tot_loyalty_sum,weights_sum,fin_loyalty
material,,,
000062284ffb915ba6b88a36e3cb7ba8,1.424395,21.25000,0.067030
00011c1252f877221e0e0a3ddf21858e,9.962606,155.25000,0.064171
000189cbec3b82946747db131d67805a,12.108905,137.80000,0.087873
000273bf6eb4ba9a2ec52c0b5ee564ed,102.706557,1424.40415,0.072105
00033dd12618c86f7451b1e18af8bb60,3.964742,87.75000,0.045182


In [151]:
mat_df.reset_index(level=0, inplace=True)
mat_df.head(5)

,material,tot_loyalty_sum,weights_sum,fin_loyalty
0,000062284ffb915ba6b88a36e3cb7ba8,1.424395,21.25000,0.067030
1,00011c1252f877221e0e0a3ddf21858e,9.962606,155.25000,0.064171
2,000189cbec3b82946747db131d67805a,12.108905,137.80000,0.087873
3,000273bf6eb4ba9a2ec52c0b5ee564ed,102.706557,1424.40415,0.072105
4,00033dd12618c86f7451b1e18af8bb60,3.964742,87.75000,0.045182


In [152]:
mat_df['norm_loyalty'] = (mat_df['fin_loyalty'] - mat_df['fin_loyalty'].min()) / (mat_df['fin_loyalty'].max() - mat_df['fin_loyalty'].min())
mat_df.sort_values('norm_loyalty', ascending=False)

,material,tot_loyalty_sum,weights_sum,fin_loyalty,norm_loyalty
101110,fab9bca8c00636e7734d4b7131ac67b2,752.604164,831.05,0.905606,1.000000
22090,374b141aa6b44d2e17188ded130fb689,5.088261,6.30,0.807660,0.891845
53713,8596002c437c9b93d9ae7bb0ae0f546b,1.867640,2.40,0.778183,0.859295
72134,b3638a21c80cc12b2a2af24f8a91515c,19.330365,24.85,0.777882,0.858962
87494,d9243ba7883e025a58586e223cbff8a0,408.018607,526.60,0.774817,0.855578
...,...,...,...,...,...
29112,48e221e87bf2ab773cdd1d81908fbe09,0.000000,0.00,0.000000,0.000000
34365,558702f54b80ed2524305a7471663099,0.000000,0.00,0.000000,0.000000
97262,f12fddba91ca864de56d19bb989358a7,0.000000,0.00,0.000000,0.000000
17276,2b81c60edd577be13f321a3ae815bd82,0.000000,0.00,0.000000,0.000000


In [153]:
len(mat_df[(mat_df['norm_loyalty'] > 0.1) & (mat_df['norm_loyalty'] < 0.5)])

31495

In [154]:
len(mat_df[mat_df['norm_loyalty'] < 0.1])

71578

In [155]:
len(mat_df[mat_df['norm_loyalty'] > 0.5])

235

In [156]:
mat_df.to_csv(f"{DATADIR}/materials_loyalty_fin.csv", index=False)

In [ ]:
mat_df.sort_values('norm_loyalty', ascending=False)

,material,tot_loyalty_sum,weights_sum,fin_loyalty,norm_loyalty
46524,738087118179891d3b03f5a8d636249d,68.390843,1.00,68.390842,1.000000
84168,d101d982d09b95a246320f69718524f2,84.730547,2.00,42.365273,0.619458
67296,a74fa23431001f4dc37e0c78b93374f2,12.574751,0.30,41.915836,0.612887
80148,c73bb523e9c3b0351151c5db4859d77d,50.421013,1.85,27.254602,0.398512
101110,fab9bca8c00636e7734d4b7131ac67b2,558.675416,26.15,21.364261,0.312385
...,...,...,...,...,...
68796,aaf6abe3e42590b507e8b02a867e2df3,0.000000,0.00,0.000000,0.000000
66992,a6896dd14fe5379c3bd02876e4816a18,0.000000,0.00,0.000000,0.000000
32688,517f427b22d6455e31d240cc743cad28,0.000000,0.00,0.000000,0.000000
10092,19600dd43a7e5b351474c60c639ff7d8,0.000000,0.00,0.000000,0.000000


In [ ]:
percentiles = [mat_df['norm_loyalty'].quantile(0.25), mat_df['norm_loyalty'].quantile(0.75)]

In [ ]:
def get_class(x, percentiles):
    if x > percentiles[1]:
        return 3
    elif x < percentiles[0]:
        return 1
    else:
        return 2

In [ ]:
mat_df['loyalty_class'] = mat_df['norm_loyalty'].apply(lambda x : get_class(x, percentiles))
mat_df.head(5)

,material,mat_loyalty_sum,weights_sum,loyalty,norm_loyalty,loyalty_class
0,000062284ffb915ba6b88a36e3cb7ba8,11.965281,27.80,0.430406,0.430406,1
1,00011c1252f877221e0e0a3ddf21858e,91.939601,96.85,0.949299,0.949299,3
2,000189cbec3b82946747db131d67805a,58.939112,121.25,0.486096,0.486096,2
3,000273bf6eb4ba9a2ec52c0b5ee564ed,922.939587,1398.90,0.659761,0.659761,2
4,00033dd12618c86f7451b1e18af8bb60,62.645833,87.85,0.713100,0.713100,2


In [ ]:
mat_df.to_csv('materials_class.csv', index=False)